In [ ]:
# import OS module
import os
import numpy as np
from matplotlib.pyplot import *
from scipy.misc import *
from skimage import color
from skimage import io
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def softmax(ar):
    ans = []
    for v in ar:
        ans.append(np.exp(v)/ sum(np.exp(ar)))
    return ans              

In [ ]:
# Get the list of all files and directories
path = "./images/"
dir_list = os.listdir(path)

In [ ]:
data = np.zeros((1, 504, 504))
data = data[1:]
data.shape

In [ ]:
for file in dir_list:
    img = color.rgb2gray(imread(path + file)[29: , :, :])
    data = np.concatenate((data,np.reshape(img, (1, 504, 504)) ))
data.shape
imshow(img, cmap="gray")

In [ ]:
data.shape

In [ ]:
labels = open("labels.txt", "r")
labels = labels.read()
labels = labels.split('\n')

In [ ]:
hour_labels = np.zeros((1))[1:]
min_labels = np.zeros((1))[1:]

for lbl in labels:
    time = lbl.split(":")
    if len(time) == 2:
        hour_labels = np.append(hour_labels, time[0])
        min_labels = np.append(min_labels, time[1])


In [ ]:
p = np.random.permutation(len(data))
X = data[p, :].astype(float)
h = hour_labels[p].astype(float)
m = min_labels[p].astype(float)

In [ ]:
train_data , test_data = X[:504, :], X[504 :, :]
train_h_lbls, test_h_lbls = h[:504], h[504 :]
train_m_lbls, test_m_lbls = m[:504], m[504 :]

In [ ]:
inputs = keras.Input(shape=(504, 504, 1))
x = layers.Conv2D(filters=8, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(13, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
model.fit(train_data, train_h_lbls, epochs=10, batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_h_lbls)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
inputs = keras.Input(shape=(504, 504, 1))
x = layers.Conv2D(filters=8, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(60, activation="softmax")(x)
model_minutes = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model_minutes.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
model_minutes.fit(train_data, train_m_lbls, epochs=10, batch_size=64)

In [ ]:
test_loss, test_acc = model_minutes.evaluate(test_data, test_m_lbls)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
def predict_time(image):
    hrs = model.predict(np.reshape(image, (1,504,504, 1)), verbose = 0).argmax(axis=1)
    mins =  model_minutes.predict(np.reshape(image, (1,504,504, 1)), verbose = 0).argmax(axis=1)
    return str(hrs[0]).zfill(2) + ":" + str(mins[0]).zfill(2)

In [ ]:
imshow(test_data[2], cmap="gray")

In [ ]:
for i in range(len(test_data)):
    t = predict_time(test_data[i])
    print( t + "\t" + str(int(test_h_lbls[i])).zfill(2) + ":" + str(int(test_m_lbls[i])).zfill(2))